In [1]:
import os
import glob
import sys
import json
import time
import copy
import torch
import click
import pickle
import numpy as np
import logging as log
import cv2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
import torch, torch.nn as nn
import segmentation_models_pytorch as sm
import scipy.stats as sps
import math
from scipy.stats import bartlett
from scipy import stats

from tqdm import tqdm
from pathlib import Path
from skimage.io import imread
from torch.autograd import Variable
from losses_pytorch.segmentation.twodim.entropy_based_losses import cross_entropy_with_logits_loss

from precode.models import UnetSm
from precode.scoring import jaccard_score
from precode.labels.wheat_segmentation import WHEAT_LABELS
from precode.net_utils import init_determenistic, batch_ids_generator


In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [2]:
!nvidia-smi

Sat Mar  5 15:52:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:09:00.0 Off |                  N/A |
| 30%   43C    P8    25W / 250W |   1684MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 36%   

In [3]:
model_seg_path = '/home/jupyter-fly_cv/Igor/Plants/13-45:24-09-21_model_plant_segmentation.bin'
model_checker_path = '/home/jupyter-fly_cv/Igor/Plants/10-27:11-02-21_model_segmentation_checker.bin'
imgs_path = '/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/'
#imgs_path = '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/'
standart_hsv_green = (53.0, 147.0, 140.0)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

infer_aug = A.Compose([ 
                        A.Resize ( height=512,
                                   width=512,
                                   interpolation=1,
                                   always_apply=False,
                                   p=1. ),
                        A.Normalize ( mean=(0.485, 0.456, 0.406),
                                      std=(0.229, 0.224, 0.225),
                                      max_pixel_value=255.0,
                                      always_apply=False,
                                      p=1.0 )
                      ])


class UnetSm(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()

        self.unet = sm.Unet(in_channels=in_channels, classes=out_channels, **kwargs)

    def forward(self, inputs):
        return self.unet(inputs)

def inverse_infer_aug(height, width):
    return A.Compose([
                       A.Resize ( height=height,
                                  width=width,
                                  interpolation=cv2.INTER_NEAREST,
                                  always_apply=False,
                                  p=1. ) 
                    ])

    
def create_model(model_file):
    model = UnetSm( out_channels=2,
                    encoder_name='efficientnet-b0' )
 
    with open(model_file, 'rb') as f:
        state = pickle.load(f)

    model.load_state_dict(state)
    model.cuda()

    return model


def torch_float(data, device):
    return Variable(torch.FloatTensor(data)).to(device)

def augmented_load(img, aug):
    auged = aug(image=img)
    aug_img = auged['image']

    aug_img = aug_img.transpose(2, 0, 1)

    return np.array([aug_img])

def infer(model, img):
    
    imgs_batch_ = augmented_load(img, infer_aug)
    imgs_batch = torch_float(imgs_batch_, torch.device('cuda'))

    logits_batch = model(imgs_batch)

    pred_masks_batch = logits_batch.argmax(dim=1)
    pred_mask = pred_masks_batch.cpu().data.numpy()#[0]
      #  print(pred_masks_batch.cpu().data.numpy().shape)

    original_size = img.shape[:2]

    original_pred_mask = inverse_infer_aug(*original_size)(image=pred_mask)['image']
    original_pred_mask = original_pred_mask.astype('uint8') * 255 

    return original_pred_mask

In [5]:
def filename_pref_png(name, pref):
    names = name.split('/')
    png_names = names[-1].split('.')
    names[-1] = png_names[0] + '.' + 'png'
    names[-1] = pref + names[-1]
    new_name = ''
    for n in names:
        if n != names[-1]:
            new_name =new_name + n + '/'
        else:
            new_name =new_name + n
    return new_name    

In [6]:
def all_jpg_img(path):
    img_names = []

    for dirname, _, filenames in os.walk(imgs_path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'jpg':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names

In [7]:
def color_checker(img):
    img_checker = infer(create_model(model_checker_path),img)
    img_checker = cv2.GaussianBlur(img_checker, (21, 21), 0)
    img_checker = 1 - np.round(img_checker/255)
    #plt.imshow(img_checker, cmap = 'gray')
    #print(img_checker)
    return img_checker

In [8]:
def seg_img(seg_model,checker_model, img):
    new_img = np.copy(img)
    mask = infer(seg_model,img)
    checker = color_checker(img)
    mask = mask * checker
    mask = cv2.GaussianBlur(mask, (11, 11), cv2.BORDER_DEFAULT)
    mask = cv2.threshold(mask,15,1,cv2.THRESH_BINARY)[1]
    #print(new_img.shape)
    for i in range(3):
        new_img[:,:,i] = mask * new_img[:,:,i]
    return new_img

In [9]:
def del_bl_sq(img):
    new_img = img[np.where(img[:,0]+img[:,1]+img[:,2] != 0),:]
    return np.reshape(new_img,(new_img.shape[1],new_img.shape[2]))

In [10]:
img_ex =np.array([[1,1,1],[0,1,1],[3,3,3],[0,0,0],[0,1,0],[1,1,0],
                  [0,0,0], [2,1,0],[2,2,2],[1,1,1],
                  [-9999,1,0],[0,9999,0],[0,0,0],[0,1,9999]])
del_img_ex = del_bl_sq(img_ex)
del_img_ex

array([[    1,     1,     1],
       [    0,     1,     1],
       [    3,     3,     3],
       [    0,     1,     0],
       [    1,     1,     0],
       [    2,     1,     0],
       [    2,     2,     2],
       [    1,     1,     1],
       [-9999,     1,     0],
       [    0,  9999,     0],
       [    0,     1,  9999]])

In [11]:
im_0 = del_img_ex[:,0]
im_0

array([    1,     0,     3,     0,     1,     2,     2,     1, -9999,
           0,     0])

In [12]:
def sigma_3(mass):
    std = mass.std()
    E = mass.mean()
    return mass[np.where((mass > E - 3*std) & (mass < E + 3*std))] 

In [13]:
def sigma_2(mass):
    std = mass.std()
    E = mass.mean()
    return mass[np.where((mass > E - 2*std) & (mass < E + 2*std))] 

In [14]:
im_0 = sigma_2(im_0)
im_0

array([1, 0, 3, 0, 1, 2, 2, 1, 0, 0])

In [15]:
def pr_hist(mass,number = 4, size = 256):
    if mass.shape[0] == 0:
        return 0,0,0,0
    hist = []
    ud_size = size / number
    for i in range(number):
        h = mass[np.where((mass >= i*(size/number)) &(mass < (i+1)*(size/number)))].shape[0]
        hist.append(h/ mass.shape[0])
    return hist                

In [16]:
pr_hist(im_0,number = 4, size = 8)

[0.7, 0.3, 0.0, 0.0]

In [17]:
def strain_contours(contours,porog_a = 0, porog_b = 999999999):
    list_cnt = []
    new_cnt = []
    area_cnt = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > porog_a and area < porog_b:
            area_cnt.append(area)
            list_cnt.append(cnt)
    
    if len(area_cnt) == 0:
        return -1
    
    mean_area = sum(area_cnt)/len(area_cnt)
    for cnt in list_cnt:
        area = cv2.contourArea(cnt)
        if area > mean_area * 0.9 and area < mean_area * 15:
            new_cnt.append(cnt)
    
    return np.array(new_cnt)

In [18]:
def green_cont(contours,img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    max_sum = 0
    green_med_0 = -1
    green_med_1 = -1
    green_med_2 = -1
    green_hsv_med = -1
    cont = -1
    
    for cnt in contours:
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        area = cv2.contourArea(cnt)
        a = int(math.sqrt(area)/4) #четверть стороны
        #print(cx,cy,a)
        median_0 = np.median(sigma_2(img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),0]))
        median_1 = np.median(sigma_2(img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),1]))
        median_2 = np.median(sigma_2(img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),2]))
        
        hsv_median_h = np.median(sigma_2(hsv_img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),0]))
        hsv_median_s = np.median(sigma_2(hsv_img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),1]))
        hsv_median_v = np.median(sigma_2(hsv_img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),2]))
        
        med_sum = (255 - median_0) + median_1 + (255 - median_2)
        
        #print(median_0,median_1,median_2)
        
        #cv2.circle(img,(cx,cy,),10,(0,255,0))
        
        if med_sum > max_sum and median_1 > median_0 + 5 and median_1 > median_2 + 5:
            max_sum = med_sum
            green_med_0 = median_0
            green_med_1 = median_1
            green_med_2 = median_2
            green_hsv_med = (hsv_median_h,hsv_median_s,hsv_median_v)
            cont = cnt
            #print(green_hsv_med)
            #cv2.imwrite('/home/jupyter-fly_cv/Igor/Plants/crop_img.png',img[int(cy-a):int(cy+a),int(cx-a):int(cx+a),:])
                    
    return green_hsv_med #wh_med_0, wh_med_1, wh_med_2

In [19]:
def normal(img,thresh = 25):
    checker = np.copy(img)
    mask_checker = infer(create_model(model_checker_path),img)
    mask_checker = cv2.GaussianBlur(mask_checker, (21, 21), 0)
    for i in range(3):
        checker[:,:,i] = np.round(mask_checker/255) * checker[:,:,i]
    bl_checker = cv2.cvtColor(checker,cv2.COLOR_BGR2GRAY)
    ret, thresh_checker = cv2.threshold(bl_checker, thresh, 255, cv2.THRESH_BINARY)
    #return thresh_checker
    contours, hierarchy = cv2.findContours(thresh_checker,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #print(contours)
    contours = strain_contours(contours,porog_a = 100,porog_b =img.shape[0]*img.shape[1]/20)
    #print(contours)
    #cv2.drawContours(checker,green_cont(contours,checker),-1,(0,255,0),5)
    #return checker
    green_med = green_cont(contours,checker)
    if green_med == -1:
        return -1
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    for i in range(3):
        hsv_img[:,:,i] = hsv_img[:,:,i].astype(np.float32)
        hsv_img[:,:,i] = hsv_img[:,:,i]*(standart_hsv_green[i]/green_med[i])
        hsv_img[:,:,i] = np.around(hsv_img[:,:,i])
        hsv_img[:,:,i] = np.where(hsv_img[:,:,i] <= 255,hsv_img[:,:,i],255)
        hsv_img[:,:,i] = hsv_img[:,:,i].astype(np.uint8)
    return cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)

In [20]:
wh = np.array([np.array([1,12,13,99,256,0,357]),
               np.array([1,12,13,99,255,0,257])])
wh = np.where(wh <= 255,wh,255)
wh.astype(np.uint8)

array([[  1,  12,  13,  99, 255,   0, 255],
       [  1,  12,  13,  99, 255,   0, 255]], dtype=uint8)

In [21]:
def mean_color_and_hist(seg_model,checker_model,img):
    img = np.copy(seg_img(seg_model,checker_model,img))
    
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_img = np.reshape(hsv_img,(hsv_img.shape[0] * hsv_img.shape[1],hsv_img.shape[2]))
    
    hsv_img = del_bl_sq(hsv_img)
    
    if hsv_img.shape[0] == 0:
        return 0, 0, 0, 0,0,0,0, 0,0,0,0,0,0,0,0
    
    print(hsv_img.shape)
    
    h = hsv_img[:,0]
    s = hsv_img[:,1]
    v = hsv_img[:,2]
    
    h = sigma_2(h)#np.array(list(filter(lambda x:sigma_3(x,E = h.mean(), std = h.std()),h)))
    s = sigma_2(s)#np.array(list(filter(lambda x:sigma_3(x,E = s.mean(), std = s.std()),s)))
    v = sigma_2(v)#np.array(list(filter(lambda x:sigma_3(x,E = v.mean(), std = v.std()),v)))
    
   # print(h.shape[0])
    
    hist_h = tuple(pr_hist(h,size = 180))
    hist_s = tuple(pr_hist(s))
    hist_v = tuple(pr_hist(v))
    
    return np.median(h), np.median(s), np.median(v), hist_h[0],hist_h[1],hist_h[2],hist_h[3], hist_s[0],hist_s[1],hist_s[2],hist_s[3],hist_v[0],hist_v[1],hist_v[2],hist_v[3]

In [22]:
def get_ploidn(tbl, name):
    name1 = name.split('-')[0]
    if len(name.split('-')) > 1:
        name2 = name.split('-')[1]
    else:
        name2 = ''
    for i in range(tbl.shape[0]):
        if name1 == tbl['Номер растения'].array[i].split('/')[0] and (name2 == tbl['Номер растения'].array[i].split('/')[1] or '0'+ name2 == tbl['Номер растения'].array[i].split('/')[1]):
                return tbl['Устьица'].array[i], tbl['хромосомы'].array[i]
    return '-', '-'        

In [23]:
def create_table_with_Mean_Color(mean_color):
    print('hey')
    df = pd.DataFrame(mean_color,columns = ['Name', 'Median_H','Median_S','Median_V',
                                        'h1','h2','h3', 'h4',
                                        's1', 's2','s3','s4',
                                        'v1', 'v2','v3','v4',
                                        'Path','Устьица','Хромосомы'])
    df.to_csv('mean_color_and_hist_median.csv', index = False)
    return df

In [24]:
def direct_filename(name):
    names = name.split('/')
    direct = ''
    for n in names:
        if n == names[-1]:
            continue
        direct = direct + n + '/' 
    return direct   

In [25]:
green = np.uint8([[[0,255,0]]])
hsv_green = cv2.cvtColor(green,cv2.COLOR_BGR2HSV)
print(hsv_green)
green_bgr = cv2.cvtColor(hsv_green,cv2.COLOR_HSV2BGR)
print(green_bgr)

[[[ 60 255 255]]]
[[[  0 255   0]]]


In [26]:
def name_1_folder(path):
    names = path.split('/')
    for name in names:
        if name == '1':
            return True
    return False

In [27]:
img_names = all_jpg_img(imgs_path)

seg_model = create_model(model_seg_path)
checker_model = create_model(model_checker_path)

mean_mass = []

annot_df = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/annotation.csv', sep = ';')

for name in img_names:
    if name_1_folder(name):
        continue
    #name == '/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/446-2.jpg'
     #name = '/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/2/2020_09_24/454-11.jpg'
    #name = '/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_05/451-04-3.jpg'
    print(name)
    img =cv2.imread(name) #normal(cv2.imread(name),thresh = 70)
    #img = cv2.imread(name)
    if type(img) == int:
        continue
    #cv2.circle(img,(int(img.shape[1]/2),(int(img.shape[0]/2))),100,(0,255,0))
#    plt.imshow(img, cmap = 'gray')
#    cv2.imwrite('/home/jupyter-fly_cv/Igor/Plants/norm_img.png',img)
    #print(cv2.imread(name))
#    break
    #print(img.shape)
    m_h = mean_color_and_hist(seg_model,checker_model,img)
    print(m_h)
    ust, chrom = get_ploidn(annot_df, name.split('/')[-1].split('.')[0])
    mean_mass.append([name.split('/')[-1].split('.')[0],
                      m_h[0],m_h[1],m_h[2],m_h[3],m_h[4],
                      m_h[5],m_h[6],m_h[7],m_h[8],
                      m_h[9],m_h[10],m_h[11],m_h[12],
                      m_h[13],m_h[14],
                      name,ust,chrom])
    os.chdir(direct_filename(name))

/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/453-4-1.jpg
(608283, 3)
(36.0, 201.0, 86.0, 0.9699209286046804, 0.0300790713953196, 0.0, 0.0, 0.0, 0.0632686441206147, 0.3869982599438398, 0.5497330959355455, 0.4108625919510645, 0.431227227072808, 0.15791018097612752, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/454-26-4.jpg
(784129, 3)
(37.0, 206.0, 77.0, 0.9438094775748304, 0.056190522425169614, 0.0, 0.0, 0.0, 0.03889964853768457, 0.3810186602214318, 0.5800816912408836, 0.4138426016656911, 0.4630527806294947, 0.1231046177048142, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/454-19.jpg
(531726, 3)
(37.0, 171.0, 91.0, 0.9588319944973553, 0.04116800550264469, 0.0, 0.0, 0.0, 0.1519402494122655, 0.48691311204202664, 0.36114663854570783, 0.3477774387142712, 0.48153799206591397, 0.17068456921981487, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/449-1-3.jpg


(285167, 3)
(36.0, 170.0, 96.0, 0.9852759560514437, 0.01472404394855634, 0.0, 0.0, 0.01667308503420073, 0.177598063486824, 0.4655407016192624, 0.3401881498597128, 0.3127651314628551, 0.46694194459149463, 0.22029292394565025, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/452-4-3.jpg
(386700, 3)
(36.0, 199.0, 72.0, 0.970145867953892, 0.02985413204610799, 0.0, 0.0, 0.0, 0.09738280985007801, 0.35882233227053795, 0.5437948578793841, 0.45027156237368804, 0.423259872744453, 0.12646856488185898, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/991-6-3.jpg
(656645, 3)
(36.0, 174.0, 84.0, 0.9658718133998431, 0.03412818660015695, 0.0, 0.0, 0.0, 0.1438448907137613, 0.4779055988554022, 0.37824951043083654, 0.3872293597239114, 0.3950854133611562, 0.21605845274329072, 0.0016267741716416528)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/451-5.jpg
(436807, 3)
(37.0, 178.0, 84.0, 0.9450905983401168, 0.054

(585867, 3)
(37.0, 185.0, 90.0, 0.9658372150036774, 0.03416278499632263, 0.0, 0.0, 0.0, 0.10490555068146755, 0.4424381881001038, 0.4526562612184286, 0.35412429495187264, 0.46659646297271345, 0.1792792420754139, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/454-23-4.jpg
(322859, 3)
(37.0, 182.0, 85.0, 0.9565835633399541, 0.043416436660045916, 0.0, 0.0, 0.0, 0.11564838005459298, 0.4642943262525359, 0.4200572936928711, 0.39186841409525247, 0.43156009618410945, 0.17657148972063805, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/991-1.jpg
(623670, 3)
(37.0, 191.0, 88.0, 0.9604148357628097, 0.03958516423719022, 0.0, 0.0, 0.0, 0.10864550203037676, 0.39991031464611654, 0.4914441833235067, 0.31312117160385966, 0.4381157412547244, 0.24703824735682783, 0.001724839784588106)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/991-1-3.jpg
(634411, 3)
(37.0, 194.0, 93.0, 0.9877324552987634, 0.012267544701

(544579, 3)
(37.0, 176.0, 92.0, 0.9696882988776652, 0.030311701122334878, 0.0, 0.0, 0.0, 0.11180178469717106, 0.5246990696791343, 0.36349914562369473, 0.34300238072775907, 0.43154806498813336, 0.22051421499802992, 0.004935339286077645)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/991-2.jpg
(624983, 3)
(40.0, 170.0, 75.0, 0.9377918579454518, 0.06220814205454821, 0.0, 0.0, 0.009450694148194678, 0.17767036274407716, 0.5415296453176488, 0.2713492977900794, 0.377991047796458, 0.5601308656700417, 0.0618780865335004, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/452-1-2.jpg
(473072, 3)
(37.0, 192.0, 82.0, 0.9773769295794953, 0.02262307042050471, 0.0, 0.0, 0.0, 0.12293507069562563, 0.37644459961836, 0.5006203296860143, 0.3597181725643105, 0.5220449957730198, 0.11823683166266973, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/452-3.jpg
(560484, 3)
(37.0, 191.0, 85.0, 0.9509209644396749, 0.0490

(405229, 3)
(12.0, 142.0, 79.0, 0.9933961333222003, 0.006603866677799622, 0.0, 0.0, 0.008030608244217016, 0.32124055321967904, 0.6320845782578718, 0.03864426027823219, 0.3745998399359744, 0.4729316726690676, 0.152468487394958, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/447-2-1.jpg
(281080, 3)
(37.0, 191.0, 81.0, 0.9490321053302496, 0.05096789466975033, 0.0, 0.0, 0.002346559622759763, 0.12114813543640786, 0.38012773640935343, 0.49637756853147896, 0.397271998636362, 0.4264814149920394, 0.17624658637159862, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/453-4-2.jpg
(605317, 3)
(37.0, 182.0, 89.0, 0.9566342552045605, 0.043365744795439566, 0.0, 0.0, 0.0, 0.09357901144346346, 0.4784619062717776, 0.42795908228475893, 0.3961209240835725, 0.4408725410624786, 0.16300653485394895, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/3/2020_10_19/454-31-2.jpg
(327674, 3)
(37.0, 179.0, 94.0, 0.957392218186179,

...

(126825, 3)
(36.0, 170.0, 92.0, 0.9660125794404769, 0.03398742055952303, 0.0, 0.0, 0.0, 0.13368001219796005, 0.5229072874786335, 0.34341270032340643, 0.43346788527011765, 0.282156201369744, 0.2810502950400101, 0.0033256183201282852)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-65-3.jpg
(91369, 3)
(37.0, 166.0, 85.0, 0.9405983392105562, 0.05933340916846775, 6.825162097599818e-05, 0.0, 0.0007528953254057884, 0.2116743063398215, 0.45313226599348966, 0.334440532341283, 0.45080357632603807, 0.27249190938511325, 0.2729416927211892, 0.003762821567659481)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-53-2.jpg
(133259, 3)
(31.0, 182.0, 61.0, 0.9637215297316839, 0.03615272167119886, 0.0001257485971172134, 0.0, 0.0, 0.141506836744058, 0.4399528922333364, 0.4185402710226056, 0.504117281988405, 0.2603651060351483, 0.23551761197644672, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-56-4.jpg


(77836, 3)
(30.0, 170.0, 58.0, 0.9450444423205582, 0.05461421881186767, 0.00034133886757417293, 0.0, 0.0025859668200564935, 0.17815322184943042, 0.46284827668518835, 0.3564125346453247, 0.5153211587712186, 0.2538898112042006, 0.14068954866607467, 0.09009948135850611)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/991-9-1.jpg
(17611, 3)
(25.0, 173.0, 74.0, 0.976751064836725, 0.023248935163274965, 0.0, 0.0, 0.0, 0.11372867587327376, 0.528838342810723, 0.35743298131600326, 0.48026801430923854, 0.1338368065413662, 0.3669865425018454, 0.018908636647549826)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-56-3.jpg
(86367, 3)
(36.0, 179.0, 79.0, 0.9535366131488388, 0.04646338685116128, 0.0, 0.0, 0.0, 0.11668041966285513, 0.48728044323942, 0.39603913709772487, 0.46402827674122143, 0.3248928033375826, 0.21107891992119596, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-49-2.jpg
(134739, 3)
(35.0,

(149713, 3)
(36.0, 182.0, 85.0, 0.9669767250144267, 0.03302327498557336, 0.0, 0.0, 0.0, 0.09482175668198987, 0.4892316309314827, 0.4159466123865274, 0.44424082539554677, 0.3702868343503098, 0.18547234025414341, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-56-1.jpg
(92549, 3)
(36.0, 181.0, 74.0, 0.9624816058771328, 0.03751839412286712, 0.0, 0.0, 0.0, 0.09894697420307096, 0.503095190888463, 0.3979578349084661, 0.4789304836017601, 0.31437365876921075, 0.20669585762902912, 0.0)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-55.jpg
(19641, 3)
(21.0, 176.0, 128.0, 0.9899830482354754, 0.01001695176452458, 0.0, 0.0, 0.0, 0.060985841636077606, 0.6049816465652857, 0.3340325117986366, 0.2840486736927855, 0.2118527569879334, 0.31979023471310014, 0.18430833460618096)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-67-1.jpg
(144400, 3)
(33.0, 199.0, 72.0, 0.9698212884791167, 0.0301787115208832

(65984, 3)
(23.0, 190.0, 26.0, 0.971009397174455, 0.028670971041360353, 0.00031963178418461933, 0.0, 0.0, 0.19652457905196186, 0.33568006408577633, 0.4677953568622618, 0.5818949401794616, 0.2086864406779661, 0.20496323529411764, 0.004455383848454636)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-67-2.jpg
(141088, 3)
(31.0, 181.0, 93.0, 0.9772219281218751, 0.022778071878124816, 0.0, 0.0, 0.0, 0.15553121577217963, 0.43176196659002053, 0.41270681763779987, 0.4008324439701174, 0.31556029882604053, 0.2645748843827819, 0.01903237282106012)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-65-2.jpg
(75100, 3)
(37.0, 174.0, 97.0, 0.976061702790994, 0.023938297209006004, 0.0, 0.0, 0.0, 0.19026720886848922, 0.4073224087718592, 0.4024103823596516, 0.35439714232399433, 0.39690240716551595, 0.24827393170367607, 0.00042651880681363795)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/447-4-4.jpg
(115685, 3)

(90837, 3)
(35.0, 153.0, 92.0, 0.9550487333889416, 0.04495126661105842, 0.0, 0.0, 0.009347765750763253, 0.27217318900915904, 0.4362697751873439, 0.28220927005273383, 0.4304083137928377, 0.2626903134185409, 0.3009016149806797, 0.005999757807941698)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/447-5-3.jpg
(130803, 3)
(36.0, 182.0, 74.0, 0.9591404979554461, 0.04085950204455391, 0.0, 0.0, 0.0, 0.12755285404003314, 0.45301342250417986, 0.419433723455787, 0.4803469274773944, 0.2902672694839146, 0.22786338192778496, 0.0015224211109060711)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/454-50-4.jpg
(165000, 3)
(34.0, 185.0, 67.0, 0.9690486082260075, 0.03095139177399252, 0.0, 0.0, 0.0, 0.11229946524064172, 0.46166532310172737, 0.4260352116576309, 0.4955223516645635, 0.2531014344324756, 0.24810256222681468, 0.003273651676146235)
/home/jupyter-fly_cv/Igor/Plants/data/color_correction_01.02.22/4/2020_10_20/451-14-4.jpg
(43310, 3)
(35.0,

In [28]:
os.chdir('/home/jupyter-fly_cv/Igor/Plants/')
mean_color_df = create_table_with_Mean_Color(mean_mass)
mean_color_df

hey


,Name,Median_H,Median_S,Median_V,h1,h2,h3,h4,s1,s2,s3,s4,v1,v2,v3,v4,Path,Устьица,Хромосомы
0,453-4-1,36.0,201.0,86.0,0.969921,0.030079,0.000000,0.0,0.000000,0.063269,0.386998,0.549733,0.410863,0.431227,0.157910,0.000000,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,гаплоид,гаплоид
1,454-26-4,37.0,206.0,77.0,0.943809,0.056191,0.000000,0.0,0.000000,0.038900,0.381019,0.580082,0.413843,0.463053,0.123105,0.000000,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,гаплоид,гаплоид
2,454-19,37.0,171.0,91.0,0.958832,0.041168,0.000000,0.0,0.000000,0.151940,0.486913,0.361147,0.347777,0.481538,0.170685,0.000000,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид
3,449-1-3,37.0,208.0,77.0,0.970198,0.029802,0.000000,0.0,0.000000,0.052289,0.359438,0.588273,0.417626,0.434510,0.147865,0.000000,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид
4,991-3-4,36.0,181.0,81.0,0.960846,0.039154,0.000000,0.0,0.000000,0.121346,0.463553,0.415101,0.412330,0.430042,0.157628,0.000000,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,454-67,29.0,179.0,90.0,0.984106,0.015894,0.000000,0.0,0.075909,0.084101,0.408729,0.431261,0.358440,0.349361,0.169919,0.122280,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид
1184,454-50-1,33.0,171.0,64.0,0.965794,0.034206,0.000000,0.0,0.000000,0.144615,0.497408,0.357977,0.499597,0.242512,0.252392,0.005499,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид
1185,454-59-3,37.0,167.0,87.0,0.952541,0.047291,0.000169,0.0,0.000000,0.144742,0.534447,0.320811,0.456401,0.300445,0.242215,0.000939,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,гаплоид,гаплоид
1186,454-52-2,34.0,180.0,80.0,0.979017,0.020983,0.000000,0.0,0.000000,0.139937,0.448412,0.411651,0.462049,0.254958,0.275191,0.007801,/home/jupyter-fly_cv/Igor/Plants/data/color_co...,диплоид,диплоид


In [29]:
annot_df = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/annotation.csv', sep = ';')
annot_df.shape

(155, 4)